# 3 - Monstrinhos

## 3.5 Forma, função e ativação

### Introdução:

O objetivo desta atividade é implementar 3 novas funções de ativação da rede neural feita em Python
puro nos vídeos da disciplina [1], descrevendo brevemente sobre estas funções de ativação e suas diferenças com relação à função de ativação sigmoidal. Além disso, mostrou-se alguns exemplos de que o código funciona rodando alguns testes simples. A fundamentação teórica sobre as funções foi baseada em 2 sites, que explicam e mostram gráficos ilustrativos [2,3].

#### Função ReLU (Unidade Linear Retificada):

Essa função transforma os dados de entrada dos neurônios zerando os valores negativos e retornando a própria entrada para valores positivos. Matematicamente:

$$ReLU(x) = max(0, x) $$

Assim, para entradas maiores que 0, o ReLU atua como uma função linear com um gradiente de 1, não alterando a escala de entradas positivas. Isso é computacionalmente barato, porque envolve um limiar simples em zero, permitindo que as redes sejam dimensionadas para muitas camadas sem um aumento significativo na carga computacional, em comparação com funções mais complexas, como a sigmoide.

No entanto, uma desvantagem desse modelo é que ele naturalmente leva a ativações esparsas, ativando apenas um subconjunto de neurônios, o que pode criar "neurônios mortos" no treinamento, com a informação de um neurônio não sendo passada adiante.

#### Função Tanh (Tangente Hiperbólica):

A função de tangente hiperbólica converte os valores em um intervalo de -1 a +1, podendo ser definida de algumas formas [4]. Para esse problema, o formato da função escolhida foi:

$$tanh(x) = \frac{e^{2x} - 1}{e^{2x} + 1}$$

Ao contrário da função sigmoide, que tem um intervalo de 0 a 1, a tanh é centrada em zero, lidando melhor com valores negativos, além de ter aprendizado e convergência mais rápidos durante o treinamento.  

Apesar dessas vantagens, a função tanh ainda pode apresentar o problema do gradiente de desaparecimento, quando os gradientes ficam muito próximos de zero, o que afeta a atualização dos parâmetros do modelo. Isso pode desacelerar o processo de treinamento, resultando em convergências ruins.

#### Função Swish:

Essa função de ativação foi desenvolvida por pesquisadores do Google, buscando resolver problemas associados à função ReLU. Matematicamente:

$$swish(x) = x * sigmoide(x)$$

Ela é uma função suave que não muda de direção abruptamente para valores próximos de 0, curvando-se levemente e voltando a subir para valores positivos. Isso melhora a expressão dos dados de entrada e dos pesos a serem atualizados, ao contrário da ReLU, que implica a perda de parte da informação passada pela rede neural. 

### Desenvolvimento:

Importando as bibliotecas necessárias [5,6]:

In [3]:
import math
import random

Definindo as classes principais da rede neural, contendo cada neurônio e camada. Essas classes foram baseadas no material de aula do Daniel Cassar [1] e do vídeo do Andrej Karpathy [7], adaptando para as funções de ativação escolhidas. Definindo as 3 funções na classe Valor:

In [4]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    # Funções de ativação
    def sig(self):
        """Função Sigmoide. Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)

    def relu(self):
        """Função ReLU. Realiza a operação: max(0, self), """
        progenitor = (self, )
        data = max(0, self.data)
        operador_mae = "relu"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_relu():
            if data > 0:
                self.grad += resultado.grad
            else:
                pass
        
        resultado.propagar = propagar_relu
        
        return resultado
        
    def tanh(self):
        """Função tangente hiperbólica. Realiza a operação: (exp(2 * self) - 1) / (exp(2 * self) + 1)"""

        x = 2 * self
        return (x.exp() - 1) / (x.exp() + 1)
   
    def swish(self):
        """Função Swish. Realiza a operação: self * """
        return self * self.sig()
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

Na classe Neuronio é possível escolher a função de ativação desejada como argumento para criar um neurônio, aplicando ao dado de saída:

In [5]:
class Neuronio:
    def __init__(self, num_dados_entrada, fc = 'Sigmoide'):
        """Modo de representar cada neurônio da rede neural
        
        Args:
            num_dados_entrada: inteiro, indica o número de dados de entrada do neurônio
            fc: string, indica a função de ativação aplicada ao neurônio, podendo ser:
                - Sigmoide (padrão)
                - ReLU
                - Tanh
                - Softmax

        Return
            Ao ser chamada retorna o valor de saída do neurônio (tipo Valor) após transformar 
            os dados de entrada com o peso, viés e função de ativação
        """
        
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
                    
    def __call__(self, x, fc):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  

        if fc == "Sigmoide":
            dado_de_saida = soma.sig() 
        elif fc == "ReLU":
            dado_de_saida = soma.relu() 
        elif fc == "Tanh":
            dado_de_saida = soma.tanh() 
        elif fc == "Swish":
            dado_de_saida = soma.swish()  

        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

As outras classes mantém-se constantes, definindo como construir as camadas e a rede neural completa, apenas incluindo a função de ativação desejada como argumento:

In [6]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada, fc):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada, fc)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x, fc):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x, fc)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [7]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada, fc):
        self.funcao = fc
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i], fc)
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x, fc):
        for camada in self.camadas:
            x = camada(x, fc)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

Definindo as etapas de treinamento da rede neural, em que fazemos a previsão do modelo, calculamos o erro, zeramos os gradientes, fazemos o backpropagation e atualizamos os parâmetros, dependendo da rede neural escolhida:

In [8]:
def treinamento(NUM_EPOCAS, TAXA_DE_APRENDIZADO, rede_neural, x, y_true):    
    for epoca in range(NUM_EPOCAS):
        # forward pass
        y_pred = []
        for exemplo in x:
            previsao = rede_neural(exemplo, rede_neural.funcao)
            y_pred.append(previsao)

        # loss
            erros = []
            for yt, yp in zip(y_true, y_pred):
                residuo = yp - yt
                erro_quadratico = residuo ** 2
                erros.append(erro_quadratico)        
            loss = sum(erros)

        # zero grad
        for p in rede_neural.parametros():
            p.grad = 0

        # backpropagation
        loss.propagar_tudo()

        # atualiza parâmetros
        for p in rede_neural.parametros():
            p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

        # mostra resultado (opcional)
        print(f'Época: {epoca}, Perda: {loss.data}')

Criando a arquitetura de rede neural variando a função de ativação e definindo os hiperparâmetros:

In [9]:
NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]
FUNCAO_ATIVACAO = ['Tanh', 'ReLU', 'Swish']  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

nn_tanh = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, FUNCAO_ATIVACAO[0])
nn_relu = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, FUNCAO_ATIVACAO[1])
nn_swish = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede, FUNCAO_ATIVACAO[2])

Criando um exemplo simples para testar a performance dos modelos, com 3 dados de entrada para cada exemplo e 1 dado de saída:

In [10]:
x = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]

y_true = [1, 0, 0.2, 0.5]

Treinando a rede neural com a função *tanh*. Definiu-se um número de épocas como 100 e uma taxa de aprendizado de 0.01

In [11]:
NUM_EPOCAS = 100
TAXA_DE_APRENDIZADO = 0.01

treino_tanh = treinamento(NUM_EPOCAS, TAXA_DE_APRENDIZADO, nn_tanh, x, y_true)
treino_tanh

Época: 0, Perda: 1.7830194450528556
Época: 1, Perda: 1.1638989983369203
Época: 2, Perda: 0.7985418517673731
Época: 3, Perda: 0.5957506923419789
Época: 4, Perda: 0.4811783443478552
Época: 5, Perda: 0.41308113281525166
Época: 6, Perda: 0.3700609320626339
Época: 7, Perda: 0.3411010848148787
Época: 8, Perda: 0.32033947398628204
Época: 9, Perda: 0.30453561783178434
Época: 10, Perda: 0.2918335293646483
Época: 11, Perda: 0.28113703629960757
Época: 12, Perda: 0.271781825725135
Época: 13, Perda: 0.26335662566049656
Época: 14, Perda: 0.25560228838155186
Época: 15, Perda: 0.24835308260682756
Época: 16, Perda: 0.2415016318525439
Época: 17, Perda: 0.23497750118702274
Época: 18, Perda: 0.22873387208266155
Época: 19, Perda: 0.2227391219393957
Época: 20, Perda: 0.21697143762786952
Época: 21, Perda: 0.2114153380374317
Época: 22, Perda: 0.2060594150277543
Época: 23, Perda: 0.20089486116080968
Época: 24, Perda: 0.1959145102086182
Época: 25, Perda: 0.19111221415423443
Época: 26, Perda: 0.18648244199427563

O treino foi bem-sucedido, diminuindo a perda a cada época

Repetindo o procedimento para o modelo com função ReLU:

In [12]:
NUM_EPOCAS = 100
TAXA_DE_APRENDIZADO = 0.5

treino_relu = treinamento(NUM_EPOCAS, TAXA_DE_APRENDIZADO, nn_relu, x, y_true)
treino_relu

Época: 0, Perda: 1.29
Época: 1, Perda: 1.29
Época: 2, Perda: 1.29
Época: 3, Perda: 1.29
Época: 4, Perda: 1.29
Época: 5, Perda: 1.29
Época: 6, Perda: 1.29
Época: 7, Perda: 1.29
Época: 8, Perda: 1.29
Época: 9, Perda: 1.29
Época: 10, Perda: 1.29
Época: 11, Perda: 1.29
Época: 12, Perda: 1.29
Época: 13, Perda: 1.29
Época: 14, Perda: 1.29
Época: 15, Perda: 1.29
Época: 16, Perda: 1.29
Época: 17, Perda: 1.29
Época: 18, Perda: 1.29
Época: 19, Perda: 1.29
Época: 20, Perda: 1.29
Época: 21, Perda: 1.29
Época: 22, Perda: 1.29
Época: 23, Perda: 1.29
Época: 24, Perda: 1.29
Época: 25, Perda: 1.29
Época: 26, Perda: 1.29
Época: 27, Perda: 1.29
Época: 28, Perda: 1.29
Época: 29, Perda: 1.29
Época: 30, Perda: 1.29
Época: 31, Perda: 1.29
Época: 32, Perda: 1.29
Época: 33, Perda: 1.29
Época: 34, Perda: 1.29
Época: 35, Perda: 1.29
Época: 36, Perda: 1.29
Época: 37, Perda: 1.29
Época: 38, Perda: 1.29
Época: 39, Perda: 1.29
Época: 40, Perda: 1.29
Época: 41, Perda: 1.29
Época: 42, Perda: 1.29
Época: 43, Perda: 1.2

Note que a perda se mantém contante ao longo das épocas, indicando que o modelo não se ajusta tão bem aos dados na atualização dos parâmetros. Isso indica que aplicar a função ReLU em todos os neurônios (inclusive na camada de saída) não é uma forma razoável de treinar a rede neural, necessitando de ajustes pelo menos no output

Por fim, fazendo para o modelo com função Swish:

In [13]:
NUM_EPOCAS = 100
TAXA_DE_APRENDIZADO = 0.5

treino_swish = treinamento(NUM_EPOCAS, TAXA_DE_APRENDIZADO, nn_swish, x, y_true)
treino_swish

Época: 0, Perda: 2.381661078280655
Época: 1, Perda: 1.92245404990186
Época: 2, Perda: 1.6942557175401296
Época: 3, Perda: 1.6439927709223596
Época: 4, Perda: 1.483701122936058
Época: 5, Perda: 1.3212241280546018
Época: 6, Perda: 1.306223539197802
Época: 7, Perda: 1.3008729677760156
Época: 8, Perda: 1.2981082486685653
Época: 9, Perda: 1.29642425326925
Época: 10, Perda: 1.295294764552123
Época: 11, Perda: 1.2944870633541503
Época: 12, Perda: 1.2938823171516112
Época: 13, Perda: 1.2934135588421944
Época: 14, Perda: 1.2930402163424055
Época: 15, Perda: 1.2927363024570275
Época: 16, Perda: 1.292484419276088
Época: 17, Perda: 1.2922724916729504
Época: 18, Perda: 1.2920918836942779
Época: 19, Perda: 1.2919362604105942
Época: 20, Perda: 1.2918008726994943
Época: 21, Perda: 1.2916820925340622
Época: 22, Perda: 1.2915771022078524
Época: 23, Perda: 1.2914836812144168
Época: 24, Perda: 1.2914000568256894
Época: 25, Perda: 1.2913247972533002
Época: 26, Perda: 1.2912567339052061
Época: 27, Perda: 1.

O algoritmo se ajustou relativamente bem aos dados, diminuindo a perda a cada época. Note, no entanto, que no meu caso a perda subiu na época 5 e voltou a diminuir, mas com uma taxa menor para as épocas finais. Isso sugere que o modelo pode ter subajustado parte dos dados, mas conseguiu ser treinado posteriormente.

É possível estimar a performance dos modelos comparando sua perda, mas seria necessário fixar os parâmetros iniciais utilizando uma semente aleatória, a fim de garantir reprodutibilidade e comparatibilidade, ficando um exercício posterior - O problema era testar novas funções de ativação em exemplos simples

### Conclusão:

Foi possível aprender mais sobre outras funções de ativação presentes nas redes neurais, aumentando meu conhecimento sobre o tema e a execução de cada etapa do código. Foi interessante analisar os materiais de referência e adaptá-los ao problema atual, melhorando habilidades de criatividade e resolução crítica. Além disso, compreendeu-se a importância do ajuste dos hiperparâmetros, como a taxa de aprendizado, que influenciam diretamente na previsão do modelo e sua eficácia.

### Referências:

[1] CASSAR, Daniel. "ATP-303 NN 4.2 - Notebook MLP.ipynb". Material de Aula, 2025.

[2] Material sobre funções de ativação. 2024. Datacamp. https://www.datacamp.com/pt/tutorial/introduction-to-activation-functions-in-neural-networks

[3] Material sobre funções de ativação. 2021. v7labs: https://www.v7labs.com/blog/neural-networks-activation-functions

[4] Formas de representar a função tangente hiperbólica. Wikipedia. https://pt.wikipedia.org/wiki/Tangente_hiperbólica

[5] Biblioteca Math. https://docs.python.org/3/library/math.html

[6] Biblioteca Random. https://docs.python.org/3/library/random.html

[7] KARPATHY, Andrej. "The spelled-out intro to neural networks and backpropagation: building micrograd". YouTube. https://youtu.be/VMj-3S1tku0.